# data fitting

In [ ]:
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import time

from copy import deepcopy
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

In [ ]:
def is_digit(x):
    if re.match("^-?\d+?\.\d*?$", x) is None:
        return False
    return True

## airfoil lift coefficient
Performace data for airfoil E387(E) taken from: M.S. Selig and B.D. McGranahan, "Wind Tunnel Aerodynamic Tests of Six Airfoils for Use on Small Wind Turbines", NREL/SR-500-34515, 2004.

In [ ]:
raw = []
with open('e387e.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if is_digit(row[0]):
            raw.append([float(x) for x in row])

data = np.array(raw)
α_data = data[3:-3,0]
Cl_data = data[3:-3,1]
plt.figure(figsize=(10,10))
plt.style.use('dark_background')
plt.rcParams.update({'font.size': 14})
plt.plot(α_data, Cl_data, 'o')
plt.xlabel('α')
plt.ylabel(r'$C_l$')
plt.show()

## cubic splines

In [ ]:
sp = interp1d(α_data, Cl_data, kind='cubic')
x_sp = np.linspace(α_data[0], α_data[-1], num=101, endpoint=True)
plt.figure(figsize=(10,10))
plt.plot(x_sp, sp(x_sp), 'g-', label='cubic splines')
plt.plot(α_data, Cl_data, 'o', label='experimental data')
plt.xlabel('α')
plt.ylabel(r'$C_l$')
plt.legend()
plt.show()

## linear fit

In [ ]:
def fun(x, a, b):
    return a*x + b

In [ ]:
popt, pcov = curve_fit(fun, α_data, Cl_data)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(α_data, fun(α_data, *popt), 'r-', label='fit: Cl(x) = %5.3fx + %5.3f' % tuple(popt))
plt.plot(α_data, Cl_data, 'o', label='experimental data')
plt.xlabel('α')
plt.ylabel(r'$C_l$')
plt.legend()
plt.show()

## comparison between interpolation and fitting

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(α_data, fun(α_data, *popt), 'r-', label='fit: Cl(x) = %5.3fx + %5.3f' % tuple(popt))
plt.plot(x_sp, sp(x_sp), 'g-', label='cubic splines')
plt.plot(α_data, Cl_data, 'o', label='experimental data')
plt.xlabel('α')
plt.ylabel(r'$C_l$')
plt.legend()
plt.show()

## quadratic fit

In [ ]:
def fun_q(x, a, b, c):
    return a*x**2 + b*x + c

In [ ]:
xdata = np.linspace(0, 4, 50)
pexa = (2.5, 1.3, 0.5)
y = fun_q(xdata, *pexa)
np.random.seed(1729)
noise_amplitude = 2.0
y_noise = noise_amplitude * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.figure(figsize=(10,10))
plt.plot(xdata, ydata, 'o', label='data')
popt, pcov = curve_fit(fun_q, xdata, ydata)
plt.plot(xdata, fun_q(xdata, *pexa), 'g-', label=r'exact: $y$ = %5.3f$x^2$ + %5.3f$x$ + %5.3f' % pexa)
plt.plot(xdata, fun_q(xdata, *popt), 'r-', label=r'fit: $\bar{y}$ = %5.3f$x^2$ + %5.3f$x$ + %5.3f' % tuple(popt))
plt.legend()
plt.show()

## Colebrook
Data taken from: C.F. Colebrook and C.M. White, "Experiments with Fluid Friction in Roughened Pipes", Proceedings of the Royal Society A, 161, 1937 (https://doi.org/10.1098/rspa.1937.0150).

In [ ]:
raw = []
with open('pipe.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if is_digit(row[0]):
            raw.append([float(x) for x in row])
data = np.array(raw)
x_data = data[:,0]
y_data = data[:,1]
plt.figure(figsize=(10,10))
plt.style.use('dark_background')
plt.plot(x_data, y_data, 'o')
plt.xlabel('Rey')
plt.ylabel(r'$\frac{8\tau}{\rho U^2}$')
plt.show()

### power law

In [ ]:
def fun_pl(x, a, b):
        return a*x**b
def fun_pl_vec(x, a, b):
        return [fun_pl(i, a, b) for i in x]

In [ ]:
x_plot = np.linspace(x_data[0], x_data[-1], 101)
popt, pcov = curve_fit(fun_pl_vec, x_data, y_data)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(20,10))

ax1.plot(x_data, y_data, 'o', label='experiments')
ax1.plot(x_plot, fun_pl_vec(x_plot, *popt), 'r-',
           label=r'power law: $\bar{{y}} = {{{0:5.3f}}}x^{{{1:5.3f}}}$'.format(popt[0], popt[1]))
ax1.set_xlabel('Rey')
ax1.set_ylabel(r'$\frac{8\tau}{\rho U^2}$')
ax1.legend()

ax0.loglog(x_data, y_data, 'o', label='experiments')
ax0.loglog(x_plot, fun_pl_vec(x_plot, *popt), 'r-',
           label=r'power law: $\bar{{y}} = {{{0:5.3f}}}x^{{{1:5.3f}}}$'.format(popt[0], popt[1]))
ax0.set_xlabel('Rey')
ax0.set_ylabel(r'$\frac{8\tau}{\rho U^2}$')
ax0.legend()
plt.show()

### exponential model

In [ ]:
def lin_fun_exp(x, a, b):
    return a + b*x
def lin_fun_exp_vec(x, a, b):
    return [lin_fun_exp(i, a, b) for i in x]
def fun_exp(x, a, b):
    return a*math.exp(b*x)

In [ ]:
x_plot = np.linspace(x_data[0], x_data[-1], 101)
popt, pcov = curve_fit(lin_fun_exp_vec, x_data, np.array([math.log(i) for i in y_data]), p0=[1e-3,-3])

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(20,10))

ax0.semilogy(x_data, y_data, 'o', label='experiments')
ax0.semilogy(x_plot, [fun_exp(i, math.exp(popt[0]), popt[1]) for i in x_plot], 'r-',
           label=r'exponential law: $\bar{{y}} = {{{0:5.3f}}}\times{{{1:0.8e}}}^x$'.format(math.exp(popt[0]), math.exp(popt[1])))
ax0.set_xlabel('Rey')
ax0.set_ylabel(r'$\frac{8\tau}{\rho U^2}$')
ax0.legend()

ax1.plot(x_data, y_data, 'o', label='experiments')
ax1.plot(x_plot, [fun_exp(i, math.exp(popt[0]), popt[1]) for i in x_plot], 'r-',
           label=r'exponential law: $\bar{{y}} = {{{0:5.3f}}}\times{{{1:0.8e}}}^x$'.format(math.exp(popt[0]), math.exp(popt[1])))
ax1.set_xlabel('Rey')
ax1.set_ylabel(r'$\frac{8\tau}{\rho U^2}$')
ax1.legend()
plt.show()

### nonlinear fitting

In [ ]:
def func(x, a, b, c, d):
        return a/(math.log10(b + c*x**d))**2
def func_vec(x, a, b, c, d):
        return [func(i, a, b, c, d) for i in x]

In [ ]:
x_plot = np.linspace(x_data[0], x_data[-1], 101)
p0 = (0.001,1.0,1.,-1.)
popt, pcov = curve_fit(func_vec, x_data, y_data, p0=p0)
plt.figure(figsize=(10,10))
plt.plot(x_data, y_data, 'o', label='data')
plt.plot(x_plot, func_vec(x_plot, *popt), 'r-',
           label=r'fit: $\bar{{y}} = \frac{{{0:5.3e}}}{{\log^2_{{10}}({{{1:5.3e}}} + {{{2:5.3e}}}x^{{{3:5.3f}}})}}$'.format(popt[0], popt[1], popt[2], popt[3]))
plt.xlabel('Rey')
plt.ylabel(r'$\frac{8\tau}{\rho U^2}$')
plt.legend()
plt.show()